### QED Installation
This notebook installs QED under /home/jovyan, located parallel to other environments. Through the use of the jupyter server proxy extension, QED can be run as a separate web-app (the underlying Jupyter server image must have serverextensions and jupyter_server_proxy both installed in the base conda environment - which is available as of r1.7.7 of nisar_ade_base).

Once installed QED can be run using the launcher, identified by the icon labelled QED. Depending on which version of QED you have installed, you may need to edit the ~/.pyre/qed.yaml file to indicate the datasets available to be viewed prior to starting QED.

Because QED is undergoing continued developmend for the addition of new features, placing the ability for users to installed and upgrade it on their own is paramount, eliminating wait times on the OnDemand support and system administration teams.

The following cell can take a while to execute (30-45 minutes). 

In [ ]:
%%bash

pip install kernda

mkdir -p ~/.local/envs
conda config --append envs_dirs ~/.local/envs

local_base="/home/jovyan/.local/envs"
env_base=${local_base}/qed
qed_tools_dir=${env_base}/tools

date_str=`date +%Y%m%d-%H%M%S`

sudo yum install -y git vim zip unzip openssh-server

# (re)create the QED environment
conda env create -f qed_support_files/qed_environment.yaml -p ${env_base} --force

# this hideousness extracts the active major.minor (only) python version
python_version=`conda run -p ${env_base} python --version | awk '/Python/ {print $2}' | awk 'BEGIN { FS="." } { print $1"."$2 }'`

# checkout, build and install QED and its dependencies
mkdir -p ${qed_tools_dir}
pushd ${qed_tools_dir}
git clone https://github.com/aivazis/mm
git clone https://github.com/pyre/pyre
git clone https://github.com/aivazis/qed
popd

mkdir -p /home/jovyan/.pyre
if [ -f /home/jovyan/.pyre/mm.pfg ]
then
     mv /home/jovyan/.pyre/mm.pfg /home/jovyan/.pyre/mm.pfg.${date_str}
fi
sed -e "s:@PREFIX@:${env_base}:g" qed_support_files/mm.pfg > /home/jovyan/.pyre/mm.pfg

mkdir -p /home/jovyan/.mm
if [ -f /home/jovyan/.mm/config.mm ]
then
     mv /home/jovyan/.mm/config.mm /home/jovyan/.mm/config.mm.${date_str}
fi
sed -e "s:@PYTHON_VERSION@:${python_version}:g" qed_support_files/config.mm > /home/jovyan/.mm/config.mm
if [ ! -f /home/jovyan/.pyre/qed.yaml ]
then
    cp qed_support_files/qed.yaml /home/jovyan/.pyre/qed.yaml
fi

# build pyre
pushd ${qed_tools_dir}/pyre
conda run -n qed python ${qed_tools_dir}/mm/mm.py --serial
popd

# build qed
pushd ${qed_tools_dir}/qed
conda run -n qed python ${qed_tools_dir}/mm/mm.py --serial
popd

mkdir -p /home/jovyan/.jupyter
if [ -f /home/jovyan/.jupyter/jupyter_notebook_config.py ]
then
    mv /home/jovyan/.jupyter/jupyter_notebook_config.py /home/jovyan/.jupyter/jupyter_notebook_config.py.${date_str}
fi
cp qed_support_files/jupyter_notebook_config.py.ops /home/jovyan/.jupyter/jupyter_notebook_config.py

# update the ipykernel to recognize the new kernel
conda run -n qed python -m ipykernel install --user --name qed --display-name "qed"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n qed kernda --env-dir ${env_base} --display-name qed -o ~/.local/share/jupyter/kernels/qed/kernel.json